In [1]:
# ----------------------------------------------------------------------
# 1. 필요한 라이브러리 및 데이터 준비
# ----------------------------------------------------------------------

import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# ----------------------------------------------------------------------
# 예시 거래 데이터 (장바구니)
# ----------------------------------------------------------------------
# 각 리스트는 하나의 거래(Transaction)를 나타냅니다.
transactions = [
    ['맥주', '기저귀', '과자'],
    ['맥주', '기저귀', '사이다'],
    ['맥주', '우유'],
    ['기저귀', '우유', '과자'],
    ['맥주', '기저귀', '과자', '우유']
]

ModuleNotFoundError: No module named 'mlxtend'

In [ ]:
# ----------------------------------------------------------------------
# 데이터 준비 및 전처리 (원-핫 인코딩)
# ----------------------------------------------------------------------

# 데이터를 DataFrame 형태로 변환 및 One-Hot Encoding
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
# 데이터를 True/False 매트릭스로 변환
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

print("--- One-Hot Encoding 데이터 (일부) ---")
print(df.head())

In [ ]:
# ----------------------------------------------------------------------
# 1.빈발 항목 집합 (Frequent Itemsets) 찾기
# ----------------------------------------------------------------------
# min_support: 최소 지지도 (0.6은 전체 5개 거래 중 3개 이상 포함을 의미)

# 최소 지지도 0.6 (전체 거래 5개 중 3개 이상 포함) 기준으로 빈발 항목 집합 찾기
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)

print("\n--- 빈발 항목 집합 (Frequent Itemsets) ---")
print(frequent_itemsets)

In [ ]:
# ----------------------------------------------------------------------
# 2. 연관 규칙 (Association Rules) 생성 및 필터링
# ----------------------------------------------------------------------
# metric="confidence": 규칙 평가 기준으로 신뢰도(confidence) 사용
# min_threshold=0.75: 최소 신뢰도 75% 이상인 규칙만 선택

# 최소 신뢰도 0.75 기준으로 연관 규칙 생성
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.75)

print("\n--- 연관 규칙 (Association Rules) ---")
# 'antecedents' (선행 항목) -> 'consequents' (후행 항목) 순으로 정렬하여 출력
rules.sort_values(by='confidence', ascending=False)

In [ ]:
# ----------------------------------------------------------------------
# 연관 규칙 테이블 컬럼 및 값 해석
# ----------------------------------------------------------------------

# 규칙 예시: (과자) -> (기저귀)
# 값 예시: support=0.6, confidence=1.0, lift=1.25

# 1. 항목 정의 (Rule Definition)
# 'antecedents': 규칙의 선행 항목 (IF 조건). 예: (과자)
# 'consequents': 규칙의 후행 항목 (THEN 결과). 예: (기저귀)

# 2. 기본 빈도 및 확률 지표 (Frequency & Probability)
# 'support': 결합 지지도. P(A ∩ B). 전체 거래 중 A와 B가 모두 포함된 비율.
#            (예: 0.6은 과자+기저귀 구매 거래가 전체의 60%라는 의미)
# 'antecedent support': 선행 항목 A의 지지도. P(A).
# 'consequent support': 후행 항목 B의 지지도. P(B).
# 'confidence': 신뢰도. P(B|A). A를 구매했을 때 B도 구매할 조건부 확률.
#               (예: 1.0은 과자를 산 고객 100%가 기저귀도 샀다는 의미)

# 3. 흥미도 및 강력함 지표 (Interest & Strength)
# 'lift': 향상도. Confidence / Consequent Support. A 구매가 B 구매 확률을 높이는 정도.
#         - 1.0보다 크면: 양의 연관성 (함께 팔림). 예: 1.25
#         - 1.0이면: 독립적 관계 (영향 없음).
#         - 1.0보다 작으면: 음의 연관성 (서로 대체재).
# 'conviction': 규칙이 틀릴 확률의 역수. 신뢰도가 1.0일 경우 'inf' (무한대).
#               (예: inf는 규칙이 단 한 번도 틀리지 않았음을 의미)
# 'leverage': 지지도와 독립성의 차이. P(A ∩ B) - P(A)P(B).
# 'zhangs_metric': -1에서 1 사이로 정규화된 규칙의 강도 및 흥미도.

# 4. 기타 지표
# 'jaccard': A 또는 B 중 A와 B가 모두 포함된 거래의 비율. P(A ∩ B) / P(A ∪ B).
# 'certainty': 신뢰도를 보정한 지표.
# 'kulczynski': P(B|A)와 P(A|B)의 평균. 규칙의 균형 잡힌 예측 능력 측정.
# 'representativity': 지지도와 유사하며 규칙의 대표성 측정.

In [ ]:
# ----------------------------------------------------------------------
# 3. 결과 정리 및 핵심 지표 해석
# ----------------------------------------------------------------------
rules_final = rules[[
    'antecedents',     # 선행 항목 (IF A)
    'consequents',     # 후행 항목 (THEN B)
    'support',         # 지지도 (A와 B가 함께 나타나는 빈도)
    'confidence',      # 신뢰도 (A를 샀을 때 B를 살 확률 P(B|A))
    'lift'             # 향상도 (A와 B가 우연 이상으로 함께 나타나는 정도)
]].sort_values(by='confidence', ascending=False)


# 💡 지표 해석 예시: (과자) -> (기저귀) 규칙
# -------------------------------------------
# support (0.6): 과자와 기저귀를 모두 산 거래가 전체의 60%
# confidence (1.0): 과자를 산 고객은 100% 기저귀도 함께 샀다.
# lift (1.25): 과자 구매가 기저귀 구매 확률을 1.25배 증가시킨다.
# - lift가 1.0보다 크면 (1.25) -> 양의 연관성 (함께 팔림)
# - lift가 1.0이면 -> 독립적 관계 (영향 없음)
# - lift가 1.0보다 작으면 -> 음의 연관성 (서로 대체재)

print("\n--- 최종 연관 규칙 (핵심 지표) ---")
print(rules_final)
